In [ ]:
# importing libraries
import pandas as pd
import numpy as np

In [ ]:
training_csv = 'train.csv'
test_csv = 'test.csv'

no_touch_train = pd.read_csv(training_csv)
no_touch_test = pd.read_csv(test_csv)

training_data = no_touch_train
testing_data = no_touch_test

training_data.head(2)


In [ ]:
no_touch_test.head(2)

In [ ]:
training_data['Name_title'] = training_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
testing_data['Name_title'] = testing_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

training_data['Title_num'] = training_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
testing_data['Title_num'] = testing_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

training_data['Title_num'] = training_data['Title_num'].replace('Mr',1)
training_data['Title_num'] = training_data['Title_num'].replace('Mrs',2)
training_data['Title_num'] = training_data['Title_num'].replace(['Miss','Mlle','Ms','Mme'],3)
training_data['Title_num'] = training_data['Title_num'].replace(['Major','Col','Capt'],4)
training_data['Title_num'] = training_data['Title_num'].replace(['Kelly, Mr. James','Master','Don','Rev','Dr','Lady','Sir','the Countess','Jonkheer','Dona'],5)
# training_data.head(1)

testing_data['Title_num'] = testing_data['Title_num'].replace('Mr',1)
testing_data['Title_num'] = testing_data['Title_num'].replace('Mrs',2)
testing_data['Title_num'] = testing_data['Title_num'].replace(['Miss','Mlle','Ms','Mme'],3)
testing_data['Title_num'] = testing_data['Title_num'].replace(['Major','Col','Capt'],4)
testing_data['Title_num'] = testing_data['Title_num'].replace(['Kelly, Mr. James','Master','Don','Rev','Dr','Lady','Sir','the Countess','Jonkheer','Dona'],5)
testing_data.head(1)

# making sex binary
training_data['Sex_binary'] = [0 if x == 'male' else 1 for x in training_data['Sex']]
# making sex binary
testing_data['Sex_binary'] = [0 if x == 'male' else 1 for x in testing_data['Sex']]

# adding age for mean and median
train_mean = training_data.Age.mean()
train_median = training_data.Age.median()

training_data["Age_imp_median"] = training_data.Age.fillna(train_median)
training_data["Age_imp_mean"] = training_data.Age.fillna(train_mean)

# adding age for mean and median
test_mean = testing_data.Age.mean()
test_median = testing_data.Age.median()

testing_data["Age_imp_median"] = testing_data.Age.fillna(test_median)
testing_data["Age_imp_mean"] = testing_data.Age.fillna(test_mean)

# cleaning cabin - moving to decks
training_data['Cabin_decks'] = training_data['Cabin'].str.replace('\d+','')
training_data['Cabin_decks'] = training_data['Cabin_decks'].replace(['NaN'],'N')

testing_data['Cabin_decks'] = testing_data['Cabin'].str.replace('\d+','')
testing_data['Cabin_decks'] = testing_data['Cabin_decks'].replace(['NaN'],'N')

In [ ]:
testing_data.head(2)

In [ ]:
testing_data.Name_title.unique()

In [ ]:
corr = testing_data.corr()
corr 

In [ ]:
from sklearn.model_selection import train_test_split

wanted_features = training_data[['SibSp','Parch','Fare','Title_num',\
            'Sex_binary','Age_imp_median']]
target = training_data.Survived

test_want = testing_data[["PassengerId",'SibSp','Parch','Fare','Title_num',\
            'Sex_binary','Age_imp_median']]

# Creating X and y
X = training_data[['SibSp','Parch','Fare','Title_num',\
            'Sex_binary','Age_imp_median']]
# X = wanted_features[['Sex_binary','Title_num']]
y = training_data.Survived

# use train_test_splie to create training and testing data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt


model = RandomForestClassifier(criterion='gini', n_estimators=700,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=1,n_jobs=-1)
model.fit(X_train,y_train)
prediction_rm=model.predict(X_test)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the Random Forest Classifier is',round(accuracy_score(prediction_rm,y_test)*100,2))
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
result_rm=cross_val_score(model,wanted_features,target,cv=10,scoring='accuracy')
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))
y_pred = cross_val_predict(model,wanted_features,target,cv=10)
sns.heatmap(confusion_matrix(target,y_pred),annot=True,fmt='3.0f',cmap="summer")
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
testdf = testing_data[["PassengerId",'SibSp','Parch','Fare','Title_num',\
            'Sex_binary','Age_imp_median']]
testdf = pd.get_dummies(testdf, columns = ['SibSp','Parch','Fare','Title_num',\
                                           'Sex_binary','Age_imp_median'],
                             prefix=['SibSp','Parch','Fare','Title_num',\
            'Sex_binary','Age_imp_median'])
testdf.head(2)

In [ ]:
# testing_data.drop('Name',axis=1,inplace=True)
testing_data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
testing_data.isin(['NaN']).any()
np.any(np.isnan(test_want))
test_want = test_want.reset_index()

In [ ]:
# testing_data.drop(testing_data.loc[testing_data['Name']=='Kelly, Mr. James'].index, inplace=True)
# testing_data.drop(testing_data.loc[testing_data['Name']=='Wilkes, Mrs. James (Ellen Needs)'].index, inplace=True)
# testing_data.drop(testing_data.loc[testing_data['Name']=='Myles, Mr. Thomas Francis'].index, inplace=True)
# testing_data.drop(testing_data.loc[testing_data['Name']=='Wirz, Mr. Albert'].index, inplace=True)
# # testing_data.drop(testing_data.loc[testing_data['Name']==].index, inplace=True)
# # testing_data.drop(testing_data.loc[testing_data['Name']==].index, inplace=True)
# # testing_data.drop(testing_data.loc[testing_data['Name']==].index, inplace=True)
# # testing_data.drop(testing_data.loc[testing_data['Name']==].index, inplace=True)

In [ ]:
test_X  = test_want.drop("PassengerId", axis=1).copy()
Y_pred_rf = model.predict(test_X)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": testing_data["PassengerId"],
        "Survived": Y_pred_rf})